In [139]:
import rasa_core
from rasa_core.agent import Agent
from bokeh.io import output_notebook
agent = Agent.load("models/dialogue_embed")
# agent = Agent.load("models/dialogue_embed_before_norm_loss")
output_notebook()

INFO:tensorflow:Restoring parameters from models/dialogue_embed/policy_0_EmbeddingPolicy/tensorflow_embedding.ckpt


Loading BokehJS ...

In [140]:
policy = agent.policy_ensemble.policies[0]
interpreter = agent.interpreter

In [141]:
domain = agent.domain


In [142]:
from rasa_core.training.structures import StoryGraph
from rasa_core.training.dsl import StoryFileReader
from rasa_core.trackers import DialogueStateTracker
from rasa_core.channels import UserMessage
import numpy as np

def embedding_from_tracker(story, bot=False):
    reader = StoryFileReader(domain, interpreter)
    story = story.split('\n')
    story_steps = reader.process_lines(story)
#     story_graph = StoryGraph(story_steps)
    tracker = DialogueStateTracker(
            1,
            domain.slots
    )
    tracker._reset()
    for step in story_steps:
        events = step.explicit_events(domain)
        for event in events:
            tracker.update(event)
    data_X = policy.featurizer.create_X([tracker], domain)

    X, slots, prev_act = policy._create_X_slots(data_X)
    all_Y_d = policy._create_all_Y_d(X.shape[1])
    all_Y_d_x = np.stack([all_Y_d for _ in range(X.shape[0])])
    dial_vec = policy.session.run(policy.dial_embed, feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})
    bot_vec = policy.session.run(policy.bot_embed, feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})

    copy_gate = policy.session.run(policy.copy_gate, feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})
    attn_prev_act_embed = policy.session.run(policy.attn_prev_act_embed,
                                             feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})
    rnn_embed = policy.session.run(policy.rnn_embed, feed_dict={policy.a_in: X,
                                           policy.b_in: all_Y_d_x,
                                           policy.c_in: slots,
                                           policy.b_prev_in: prev_act})

    del tracker
#     print(np.shape(dial_vec))
    if bot is False:
        return (dial_vec[0,-2,:],
                copy_gate[0,-2,:],
                attn_prev_act_embed[0,-2,:],
                rnn_embed[0,-2,:])
    else:
        return bot_vec
    
def make_prediction(story):
    dial, _,_,_ = embedding_from_tracker(story, bot=False)
    bot = embedding_from_tracker(story, bot=True)[0,-2,:,:]
    abs_vals = np.array([np.absolute(vec) for vec in bot])
    norms = np.apply_along_axis(np.linalg.norm, 1, bot)
    return domain.action_for_index(np.argmax(np.dot(bot, dial)/norms)).name()
#     print(dial.shape, bot.shape)

In [143]:
# print(domain.intents)
story_block = """* request_hotel
    - utter_ask_details
* explain
    - utter_explain_details_hotel
    - utter_ask_details
* chitchat
    - utter_chitchat
    - utter_ask_details
* inform{"enddate": "May 26th"}
    - slot{"enddate": "May 26th"}
    - utter_ask_startdate
* inform{"startdate": "next week"}
    - slot{"startdate": "next week"}
    - utter_ask_location
* explain
    - utter_explain_location_hotel
    - utter_ask_location
* inform{"location": "rome"}
    - slot{"location": "rome"}
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* explain
    - utter_explain_price_hotel
    - utter_ask_price"""
# * inform{"price": "expensive"}
#     - utter_ask_people
# * inform{"people": "4"}
#     - utter_filled_slots
#     - action_search_hotel
#     - slot{"hotel": "hotel"}
#     - utter_suggest_hotel
# * chitchat
#     - utter_chitchat
#     - utter_suggest_hotel
# * affirm
#     - utter_happy"""
print(len(story_block.split('\n')))
cool_list = []
lines = story_block.split('\n')
for idx, line in enumerate(lines):
    if idx == len(lines)-1:
        continue
    if '*' in lines[idx+1] or 'slot{' in lines[idx+1]:# or 'slot{' in line:
        continue
    sub_story = '\n'.join(lines[:idx+1])
    (dial, copy, attn, rnn
    ) = embedding_from_tracker("#generate story\n{}".format(sub_story))
    pred = make_prediction("#generate story\n{}".format(sub_story))
    cool_list.append((lines[idx+1], dial, pred, copy, attn, rnn))

print(len(cool_list))
#     break
# dial = embedding_from_tracker("#generate story\n{}".format(story_block)
# )
# print(dial.shape)
# print(len(story_block.split('\n')))

35
20


In [144]:
from sklearn.decomposition import PCA

In [145]:
pca = PCA(n_components=2)
X = pca.fit_transform([ting[1] for ting in cool_list])

In [146]:
from bokeh.plotting import figure, show
import matplotlib.pyplot as plt

In [147]:
embed_dim = cool_list[0][1].shape[0]

In [148]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, Arrow, OpenHead

hover = HoverTool(tooltips=[("true action","@desc"), ("prediction", "@pred"),
                            ("copy_gate", "@copy")],names=['circle'])
p = figure( tools=[hover])

for i in range(1, len(X)):
    p.add_layout(Arrow(end=OpenHead(line_color="firebrick", size=10, line_width=2),
                       x_start=X[i-1,0], y_start=X[i-1,1],
                       x_end=X[i,0], y_end=X[i,1], line_color='gray'))

source = ColumnDataSource(data=dict(
    x=X[:,0],
    y=X[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2] for ting in cool_list],
    copy = [ting[3] for ting in cool_list]
))
p.circle('x','y', source=source, size=10, name='circle')
# p.line(X[:,0], X[:,1])

X1 = pca.transform([ting[4] for ting in cool_list])
source = ColumnDataSource(data=dict(
    x=X1[:,0],
    y=X1[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2] for ting in cool_list],
    copy = [ting[3] for ting in cool_list]
))
p.circle('x','y', source=source, size=5, color='green', name='circle')

X2 = pca.transform([ting[5] for ting in cool_list])
source = ColumnDataSource(data=dict(
    x=X2[:,0],
    y=X2[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2] for ting in cool_list],
    copy = [ting[3] for ting in cool_list]
))
p.circle('x','y', source=source, size=5, color='red', name='circle')

x = pca.transform(np.zeros((1,embed_dim)))
p.circle(x[0,0], x[0,1], size=5, color='magenta')
p.circle(X[0,0], X[0,1], size=10, color='green')
p.circle(X[-1,0], X[-1,1], size=10, color='red')


GlyphRenderer(id='f0103fd2-cd2c-4640-9519-6ab6f1529d63', ...)

In [149]:
show(p)

In [67]:
domain.index_for_action('utter_ask_price')
domain.index_for_action('utter_suggest_hotel')
domain.index_for_action('action_search_hotel')

41

In [150]:
utter_price = embedding_from_tracker("#generate story\n{}".format(story_block), bot=True)[0,-1,5,:]
utter_price.shape

(20,)

In [151]:
print(utter_price.shape)
x = pca.transform(utter_price.reshape(1,-1))
p.circle(x[0][0], x[0][1], color='black')

(20,)


GlyphRenderer(id='09076caa-24fd-4293-86d8-4aac74b012ea', ...)

In [152]:
show(p)